<a href="https://colab.research.google.com/github/bbishop3/helloAI/blob/main/colab/metaltest1_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import time
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

# Define linear model we will use below
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

def train(dataloader, model, loss_fn, optimizer, device):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test(dataloader, model, loss_fn, device):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

# modified from https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html
# get our competing devices ready ... go ahead and init all three here, but ONLY USE ONE during each test
# scroll down below and replace the references to gpu_, tpu_, cpu with whichever device you are testing
# make sure you replace ALL of them
gpu_device = torch.device("cuda")
tpu_device = torch.device("xla")
cpu_device = torch.device("cpu")

# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

# first hyperparam
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

# Show some sample data
for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break


# sample code for working with m1
  # Create a Tensor directly on the mps device
  #x = torch.ones(5, device=mps_device)
  # Or
  #x = torch.ones(5, device="mps")
  # Any operation happens on the GPU
  #y = x * 2

  # Move your model to mps just like any other device
model = NeuralNetwork().to(cpu_device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)
print("starting timer for training using CPU...")
start = time.time()
epochs = 30 
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer, cpu_device)
print(f"completed training in ... {time.time()-start}s")

print("starting timer for testing using CPU...")
start = time.time()
test(test_dataloader, model, loss_fn, cpu_device)
print(f"completed testing in ... {time.time()-start}s")

100%|██████████| 26421880/26421880 [00:01<00:00, 16452066.11it/s]


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 303193.66it/s]


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:00<00:00, 5541611.33it/s] 


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 13038814.61it/s]


Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64
starting timer for training using CPU...
Epoch 1
-------------------------------
loss: 2.312174  [   64/60000]
loss: 2.301783  [ 6464/60000]
loss: 2.277236  [12864/60000]
loss: 2.263490  [19264/60000]
loss: 2.259455  [25664/60000]
loss: 2.215531  [32064/60000]
loss: 2.229321  [38464/60000]
loss: 2.194198  [44864/60000]
loss: 2.190401  [51264/60000]
loss: 2.161554  [57664/60000]
Epoch 2
-------------------------------
loss: 2.174170  [   64/60000]
loss: 2.169331  [ 6464/60000]
loss: 2.110966  [12864/60000]
loss: 2.118908  [19264/60000]
loss: 2.083627  [25664/60000]
loss: 2.018769  [32064/60000]
loss: 2.043723  [38464/60000]
loss: 1.974871  [44864/60000]
loss: 1.966372  [51264/60000]
loss: 1.903933  [57664/60000]
Epoch 3
-------------------------------
loss: 1.941159  [   64/60000]
loss: 1.919578  [ 6464/60000]


In [1]:
import torch
import time
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

# Define linear model we will use below
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

def train(dataloader, model, loss_fn, optimizer, device):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test(dataloader, model, loss_fn, device):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

# modified from https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html
# get our competing devices ready ... go ahead and init all three here, but ONLY USE ONE during each test
# scroll down below and replace the references to gpu_, tpu_, cpu with whichever device you are testing
# make sure you replace ALL of them
gpu_device = torch.device("cuda")
tpu_device = torch.device("xla")
cpu_device = torch.device("cpu")

# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

# first hyperparam
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

# Show some sample data
for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break


# sample code for working with m1
  # Create a Tensor directly on the mps device
  #x = torch.ones(5, device=mps_device)
  # Or
  #x = torch.ones(5, device="mps")
  # Any operation happens on the GPU
  #y = x * 2

  # Move your model to mps just like any other device
model = NeuralNetwork().to(cpu_device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)
print("starting timer for training using CPU...")
start = time.time()
epochs = 30 
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer, cpu_device)
print(f"completed training in ... {time.time()-start}s")

print("starting timer for testing using CPU...")
start = time.time()
test(test_dataloader, model, loss_fn, cpu_device)
print(f"completed testing in ... {time.time()-start}s")

100%|██████████| 26421880/26421880 [00:00<00:00, 146011092.29it/s]


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 95594503.91it/s]

Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw




100%|██████████| 4422102/4422102 [00:00<00:00, 58650518.93it/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 21357346.18it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64
starting timer for training using CPU...
Epoch 1
-------------------------------
loss: 2.304855  [   64/60000]
loss: 2.301004  [ 6464/60000]
loss: 2.277327  [12864/60000]
loss: 2.276442  [19264/60000]
loss: 2.243793  [25664/60000]
loss: 2.214584  [32064/60000]
loss: 2.227688  [38464/60000]
loss: 2.186156  [44864/60000]
loss: 2.194080  [51264/60000]
loss: 2.156647  [57664/60000]
Epoch 2
-------------------------------
loss: 2.162164  [   64/60000]
loss: 2.162746  [ 6464/60000]
loss: 2.102520  [12864/60000]
loss: 2.122948  [19264/60000]
loss: 2.065418  [25664/60000]
loss: 2.003555  [32064/60000]
loss: 2.040569  [38464/60000]
loss: 1.953958  [44864/60000]
loss: 1.969145  [51264/60000]
loss: 1.897242  [57664/60000]
Epoch 3
-------------------------------
loss: 1.925929  [   64/60000]
loss: 1.904673  [ 6464/60000]
loss: 1.785973  [12864/60000]
loss: 1.830176  [19264/60000]
loss: 1.713781  [25664/60

In [2]:
import torch
import time
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

# Define linear model we will use below
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

def train(dataloader, model, loss_fn, optimizer, device):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test(dataloader, model, loss_fn, device):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

# modified from https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html
# get our competing devices ready ... go ahead and init all three here, but ONLY USE ONE during each test
# scroll down below and replace the references to gpu_, tpu_, cpu with whichever device you are testing
# make sure you replace ALL of them
gpu_device = torch.device("cuda")
tpu_device = torch.device("xla")
cpu_device = torch.device("cpu")

# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

# first hyperparam
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

# Show some sample data
for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break


# sample code for working with m1
  # Create a Tensor directly on the mps device
  #x = torch.ones(5, device=mps_device)
  # Or
  #x = torch.ones(5, device="mps")
  # Any operation happens on the GPU
  #y = x * 2

  # Move your model to mps just like any other device
model = NeuralNetwork().to(cpu_device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)
print("starting timer for training using CPU...")
start = time.time()
epochs = 30 
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer, cpu_device)
print(f"completed training in ... {time.time()-start}s")

print("starting timer for testing using CPU...")
start = time.time()
test(test_dataloader, model, loss_fn, cpu_device)
print(f"completed testing in ... {time.time()-start}s")

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64
starting timer for training using CPU...
Epoch 1
-------------------------------
loss: 2.300809  [   64/60000]
loss: 2.287689  [ 6464/60000]
loss: 2.272648  [12864/60000]
loss: 2.264005  [19264/60000]
loss: 2.251430  [25664/60000]
loss: 2.226089  [32064/60000]
loss: 2.225852  [38464/60000]
loss: 2.193606  [44864/60000]
loss: 2.192010  [51264/60000]
loss: 2.158163  [57664/60000]
Epoch 2
-------------------------------
loss: 2.169783  [   64/60000]
loss: 2.155784  [ 6464/60000]
loss: 2.099838  [12864/60000]
loss: 2.115230  [19264/60000]
loss: 2.067963  [25664/60000]
loss: 2.008293  [32064/60000]
loss: 2.030152  [38464/60000]
loss: 1.953758  [44864/60000]
loss: 1.962990  [51264/60000]
loss: 1.880919  [57664/60000]
Epoch 3
-------------------------------
loss: 1.920117  [   64/60000]
loss: 1.886402  [ 6464/60000]
loss: 1.769286  [12864/60000]
loss: 1.813733  [19264/60000]
loss: 1.706238  [25664/60